In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
#test_data.head()
#test_data.count()
print(test_data)

     PassengerId  Pclass                                          Name  \
0            892       3                              Kelly, Mr. James   
1            893       3              Wilkes, Mrs. James (Ellen Needs)   
2            894       2                     Myles, Mr. Thomas Francis   
3            895       3                              Wirz, Mr. Albert   
4            896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
..           ...     ...                                           ...   
413         1305       3                            Spector, Mr. Woolf   
414         1306       1                  Oliva y Ocana, Dona. Fermina   
415         1307       3                  Saether, Mr. Simon Sivertsen   
416         1308       3                           Ware, Mr. Frederick   
417         1309       3                      Peter, Master. Michael J   

        Sex   Age  SibSp  Parch              Ticket      Fare Cabin Embarked  
0      male  34.5      0      0 

In [4]:
# Exploring dataset...
# Classifying Cabin (0 or 1)...
train_data['Cabin'].unique() # Passengers either have a unique cabin number or none at all.
train_data['Cabin'].isna().sum() # 687 Passengers have no cabins, leaving 204 with cabins.
train_data['Cabin'] = train_data['Cabin'].notnull().astype('int') # YesCabin = 1, NoCabin = 0

# Filling NaN Age...
train_data.count() # Notice missing count of 177 Age (NaN), and 2 Embarked (NaN) data from the total 891.
train_data['Age'].unique()
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median()) # fill the Age row with median age.
train_data['Age'].isna().sum()

# Filling NaN Embarked...
train_data['Embarked'] = train_data['Embarked'].fillna('C') # fill the Embarked row with arbitrary C.
train_data['Embarked'].isna().sum()

# Binning Fare...
quantile_list = [0, .33, .67, 1.]
quantiles = train_data['Fare'].quantile(quantile_list)

quantile_labels = ['0-33Q', '33-67Q', '67-100Q']
train_data['Fare_quantile_labels'] = pd.qcut(
                                train_data['Fare'], 
                                q=quantile_list,       
                                labels=quantile_labels)
train_data.head()
print(quantiles) # 33% = 8.61295, 67% = 26.25000

0.00      0.00000
0.33      8.61295
0.67     26.25000
1.00    512.32920
Name: Fare, dtype: float64


In [5]:
# Clean Data for the Test Set...
test_data['Age'].isna().sum() # Realize 86 people with NaN age
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].median())
# test_data['Cabin'].isna().sum()
# test_data['Embarked'].isna().sum()

# Clean Cabin values...
test_data['Cabin'] = test_data['Cabin'].notnull().astype('int') # YesCabin = 1, NoCabin = 0

# Bin test data Fare also...
# quantiles: 33% = 8.61295, 67% = 26.25000
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median()) # Fill the NaN Fare
# ok nvm just cheat first.
quantile_list = [0, .33, .67, 1.]
quantiles = test_data['Fare'].quantile(quantile_list)

quantile_labels = ['0-33Q', '33-67Q', '67-100Q']
test_data['Fare_quantile_labels'] = pd.qcut(
                                test_data['Fare'], 
                                q=quantile_list,       
                                labels=quantile_labels)
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_quantile_labels
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,0,Q,0-33Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,0,S,0-33Q
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,0,Q,33-67Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,0,S,33-67Q
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,0,S,33-67Q


In [6]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [7]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


In [8]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch", "Age", "Embarked", "Fare_quantile_labels", "Cabin"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
